### 3.2

In [63]:
import osmium

class BoundaryHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.nodes = []

    def node(self, n):
        self.nodes.append((n.location.lat, n.location.lon))

handler = BoundaryHandler()
handler.apply_file("groningen-boundary.osm")
nodes = handler.nodes

In [64]:
import numpy as np

latitudes = [node[0] for node in nodes]
longitudes = [node[1] for node in nodes]

center_lat = np.mean(latitudes)
center_lon = np.mean(longitudes)

In [65]:
import folium

# Create a map centered at the calculated center
map_center = [center_lat, center_lon]
m = folium.Map(location=map_center, zoom_start=10)

# Add the boundary as a polyline
folium.PolyLine(nodes, color="blue", weight=2.5, opacity=1).add_to(m)
m.save("groningen_boundary_map.html")

In [ ]:
import networkx as nx
import folium
from geopy.distance import geodesic

# Initialize graph and add nodes with edges based on proximity
G = nx.Graph()
for i, node_a in enumerate(nodes):
    for j, node_b in enumerate(nodes):
        if i < j:  # Avoid duplicate edges
            distance = geodesic(node_a, node_b).meters
            G.add_edge(i, j, weight=distance)

# Start from the first node
visited = {0}
boundary_path = [nodes[0]]
current_node = nodes[0]

# Nearest neighbor traversal
while len(visited) < len(nodes):
    # Find the nearest unvisited node
    nearest_node = None
    nearest_distance = float('inf')
    
    for i in range(len(nodes)):
        if i not in visited:
            distance = geodesic(current_node, nodes[i]).meters
            if distance < nearest_distance:
                nearest_distance = distance
                nearest_node = i
    
    # Move to the nearest node
    boundary_path.append(nodes[nearest_node])
    visited.add(nearest_node)
    current_node = nodes[nearest_node]

# Create the map and plot the ordered boundary path
map_center = [center_lat, center_lon]
m = folium.Map(location=map_center, zoom_start=10)
folium.PolyLine(boundary_path, color="blue", weight=2.5, opacity=1).add_to(m)

# Save the map
m.save("groningen_nearest_neighbor_boundary.html")

In [57]:
from geopy.distance import geodesic

total_length = sum(geodesic(nodes[i], nodes[i + 1]).km for i in range(len(nodes) - 1))
print(f"Total boundary length: {total_length:.2f} km")

Total boundary length: 5799.16 km


### 3.2.3

In [58]:
## IN DOC 1.2 in tutorial 1 folder